In [ ]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import pandas as pd
import numpy as np
import pickle
import math
import sys

In [ ]:
seed = 99

# Load Model

In [ ]:
train_dataset = pd.read_csv("/home/admin2/ML_NIC/datasets/statlog+landsat+satellite/sat.trn", header=None,
                           sep=' ')
test_dataset = pd.read_csv("/home/admin2/ML_NIC/datasets/statlog+landsat+satellite/sat.tst", header=None,
                          sep=' ')

In [ ]:
train_features = train_dataset.drop(columns=[36])
train_labels = train_dataset[36]

In [ ]:
test_features = test_dataset.drop(columns=[36])
test_labels = test_dataset[36]

In [ ]:
dataset_scaler = MinMaxScaler(feature_range=(0, 1), clip=True)
train_features = dataset_scaler.fit_transform(train_features)
test_features = dataset_scaler.transform(test_features)

In [ ]:
# Load up model
with open("model.pkl", 'rb') as f:
    clf = pickle.load(f)

# Build Micro-C Model

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
values = clf.tree_.value

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True

print(
    "The binary tree structure has {n} nodes and has "
    "the following tree structure:\n".format(n=n_nodes)
)
for i in range(n_nodes):
    if is_leaves[i]:
        print(
            "{space}node={node} is a leaf node with value={value}.".format(
                space=node_depth[i] * "\t", node=i, value=values[i]
            )
        )
    else:
        print(
            "{space}node={node} is a split node with value={value}: "
            "go to node {left} if X[:, {feature}] <= {threshold} "
            "else to node {right}.".format(
                space=node_depth[i] * "\t",
                node=i,
                left=children_left[i],
                feature=feature[i],
                threshold=threshold[i],
                right=children_right[i],
                value=values[i],
            )
        )

In [ ]:
# Decide number of bits required for the path storage
num_features = clf.n_features_in_
classes = clf.classes_
num_leaves = int(np.sum(is_leaves))
num_result_words = math.ceil(num_leaves/32.0)
result_init = hex((2**(num_leaves % 33))-1)
features_per_result_core = int(32.0/num_result_words)
num_result_cores = [math.ceil(num_features / features_per_result_core)]

while(num_result_cores[-1] != 1):
    num_result_cores.append(math.ceil(num_result_cores[-1] / features_per_result_core))


In [ ]:
total_bits = 16
dec_bits = 13

def num_translate(number, total_bits, dec_bits):
    ret = round(number * (2**dec_bits))
    return ret
    

In [ ]:
print("__lmem uint32_t path_class[] = {", end='')
for i in range(is_leaves.shape[0]):
    if (is_leaves[i]):
        print(f"{classes[np.argmax(values[i][0])]}", end=', ')
print('};')

In [ ]:
# Depth-first search through nodes
def depth_first(root, left, right, prediction, feature, threshold, output, feature_id, num_result_words, feat_file):
    if (left[root] == right[root]):  # leaf node is base case
        if (output == ''):
            return
        
        if feat_file:
            path_id = np.sum(is_leaves[:root])
            output = output.lstrip("&& ")
            output = output.rstrip(" ")

            if (num_result_words > 1):
                result_index = num_result_words - int(path_id/32) - 1
                path_id = path_id % 32
                feat_file.write(f"        if (!({output}))\n            result_gpr[{result_index}] &= ~(1 << {path_id});\n")
            else:
                feat_file.write(f"        if (!({output}))\n            result_gpr &= ~(1 << {path_id});\n")
            feat_file.write("\n")
            
        else:
            path_id = np.sum(is_leaves[:root])
            output = output.lstrip("and ")
            output = output.rstrip(" ")

            if (num_result_words > 1):
                result_index = num_result_words - int(path_id/32) - 1
                path_id = path_id % 32
                print(f"    if (not({output})):\n        result_gpr[{result_index}] &= ~(1 << {path_id})")
            else:
                print(f"    if (not({output})):\n        result_gpr &= ~(1 << {path_id})")
            print("")
            
    else:
        
        if feat_file:
            net_threshold = num_translate(threshold[root], total_bits, dec_bits)
            if (feature[root] == feature_id):
                left_output = output + f"&& feature <= {net_threshold} "
                right_output = output + f"&& feature > {net_threshold} "
            else:
                left_output = output
                right_output = output

            depth_first(left[root], left, right, prediction, feature, threshold, 
                        left_output, feature_id, num_result_words, feat_file)
            depth_first(right[root], left, right, prediction, feature, threshold, 
                        right_output, feature_id, num_result_words, feat_file)
            
        else:
            net_threshold = num_translate(threshold[root], total_bits, dec_bits)
            if (feature[root] == feature_id):
                left_output = output + f"and feature <= {net_threshold} "
                right_output = output + f"and feature > {net_threshold} "
            else:
                left_output = output
                right_output = output

            depth_first(left[root], left, right, prediction, feature, threshold, 
                        left_output, feature_id, num_result_words, feat_file)
            depth_first(right[root], left, right, prediction, feature, threshold, 
                        right_output, feature_id, num_result_words, feat_file)

        

In [ ]:
for feat_id in range(num_features):
    print(f"def feature{feat_id+1}(feature):")
    
    if (num_result_words > 1):
        print("    result_gpr = [0xffffffff", end='')
        for i in range(1, num_result_words):
            print(", 0xffffffff", end='')
        print(']')
    else:
        print(f"    result_gpr = {result_init}")
        
    depth_first(0, children_left, children_right, values, feature, threshold, "", feat_id, num_result_words, None)
    print("    return result_gpr")
    print()
    print()

# Test Micro-C Model (Python Version)

In [ ]:
def feature1(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 896)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature > 896)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 896 and feature <= 1408)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 896 and feature <= 1408)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature > 896 and feature > 1408)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature > 896 and feature > 1408)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 896 and feature > 1408)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 1600)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 1600)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 4928)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 4928)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 4928)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 4928)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 4928)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 4928)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 4928)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 4928)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 4928)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 4928)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 4736)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 4736)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 6080)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 6080)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 6080)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 6080)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 6080)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 6080)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 6080)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 6080)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 6080)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 6080)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 6080)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 6080)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 6080)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 6080)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 6080)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 6080)):
        result_gpr[2] &= ~(1 << 25)

    return result_gpr


def feature2(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4506)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 4506)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 4506)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 3165)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 3165)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 5101)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 5101)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 3649)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 3649)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 3649)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 3649)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 3649)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 3649)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 3649)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 3649)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 2830)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 2830)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 2830)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 2830)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 2830)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 2830)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 2830)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 2830)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 2830)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 2830)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 2830)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 2830)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 4692)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 4692)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 3910)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 3910)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature <= 4245)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 4245)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 4245)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 4245)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 4245)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 5697 and feature <= 5325)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 5697 and feature <= 5325)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 5697 and feature > 5325)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 5697)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 5697)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 5697)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 5697)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 5697)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 5697)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 5697)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature <= 5697)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 5697)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 5697)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 5697)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 5697)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 5697)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 5697)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 5697)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 5697)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 5697)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 5697)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 5697)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 5697)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 5697)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 5697)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 5697)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 5697 and feature <= 5809)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 5697 and feature > 5809)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 5623)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 5623)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 5623)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 5623)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 5623)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 5623)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 5623)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 5623)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 5623)):
        result_gpr[2] &= ~(1 << 22)

    return result_gpr


def feature3(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1707)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 1707)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 1707)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature > 1707)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 1707)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 2877)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 2877)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature <= 2877)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 2877)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 2877)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 2877)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 2877)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 2877)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 2877)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 2877)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 2877)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 2877)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 2877)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 2877)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 2877)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 2877)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 3755)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 3755)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 3755)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 3755)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 3755)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 3755)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 3950)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 3950)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 3950)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 3950)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature <= 3950)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 3950)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 3950)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 3950)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 3950)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature > 3950)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 3950)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature > 3950)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 3950)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 3950)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 3950)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 3950)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 3950)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 3950)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 3950)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 3950 and feature <= 6095)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 3950 and feature <= 6095)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 3950 and feature <= 6095)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 3950 and feature > 6095)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 3950)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 3950)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 3950)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 3950)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 3950)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 3950)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 3950)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 3950)):
        result_gpr[4] &= ~(1 << 22)

    return result_gpr


def feature4(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1862)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature > 1862)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 2505)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 2505)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 2539)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 2539)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 2539)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature <= 2539)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 2539)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 2539)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 2539)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 2539)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 2539)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 2539)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 2539)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 2539)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 2539)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 2539)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 2539)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 2539)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 2539)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 2539)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 2539)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 2539)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 2539)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 2539)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 2539)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 2539)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 2539)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 2539)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 3554)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 3554)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 3554)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 3385)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 3385)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 4638)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 4638)):
        result_gpr[4] &= ~(1 << 24)

    return result_gpr


def feature5(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3501)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 3501)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 3501)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature <= 3237)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 3237)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature > 3237)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 3237)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 3237)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 3237)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 3369)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 3369)):
        result_gpr[5] &= ~(1 << 0)

    return result_gpr


def feature6(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3500)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 3500)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 3500)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 3500)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 3500)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 4022)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 4022)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 3798)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 3798)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 3798)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 1638)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature > 1638)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 1638)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 4506)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature <= 4506)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 4506)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 4506)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 4506)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 5250)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 5250)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 5250)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 5250)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 5623)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 5623)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 5623)):
        result_gpr[4] &= ~(1 << 16)

    return result_gpr


def feature7(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3234)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 3234)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 3234)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 3234)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 3234)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 3234)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 3234)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 3234)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 4355)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 4355)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 3320)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 3320)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 3320)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 5734)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 5734)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 5734)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 5260)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 5260)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 4570)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 4570)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 4570)):
        result_gpr[2] &= ~(1 << 22)

    return result_gpr


def feature8(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1344)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 1344)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 1344)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 1344)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 1344)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 4352)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 4352)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 4352)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature > 4352)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 4352)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 4096)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 3296)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 3296)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 3296)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature <= 3072)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature <= 3072)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 3072)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 3072)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 3072)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 3072)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 3072)):
        result_gpr[3] &= ~(1 << 11)

    return result_gpr


def feature9(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 448)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature > 448)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 3264)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 3264)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 3264)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 3264)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 4416)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature <= 4416)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 4416)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 2816)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 2816)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 5888)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 5888)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 5504)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 5504)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature > 5504)):
        result_gpr[6] &= ~(1 << 13)

    return result_gpr


def feature10(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2267)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 2267)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 2267)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 2267)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 2267)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 2267)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 2267)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 4494)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 4494)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 4494)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 4494)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 4494)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 4494)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 4494)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 4494)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 4494 and feature <= 5448)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 4494 and feature <= 5448)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 4494 and feature > 5448)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature <= 5090)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 5090)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 3142 and feature <= 1949)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 3142 and feature <= 1949)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 3142 and feature <= 1949)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 3142 and feature <= 1949)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 3142 and feature <= 1949)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 3142 and feature > 1949)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 3142)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 3142)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 3142)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 3142)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 3142)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 3142)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 3142)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature > 3142)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 3142)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 3142)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 3142)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 3142)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 3142)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 3142)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 3142)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 3142)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 3142)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 3142 and feature <= 5369 and feature <= 3857)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 3142 and feature <= 5369 and feature > 3857)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 3142 and feature <= 5369)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 3142 and feature <= 5369)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 3142 and feature <= 5369)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 3142 and feature > 5369)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 3142)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature > 3142)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 3142)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 3142)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 3142)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 3142)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 3142)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 3142)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 3142 and feature <= 3897)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 3142 and feature <= 3897)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 3142 and feature > 3897)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 3142 and feature > 3897)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 3142 and feature > 3897)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 3142 and feature > 3897)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 5766)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 5766)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 5528)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 1)

    return result_gpr


def feature11(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1983)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature > 1983)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 4570)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature <= 4570)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 4570)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 4570)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 4570)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 4570)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature <= 4570)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 4570)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 4570)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 4570)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 4570)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 4570)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 4570)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 4570)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 4570)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 4570)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 4570)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 4570 and feature <= 4743)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 4570 and feature > 4743)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 4570)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 4570)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 4355)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 4355)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 4355)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 4355)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 4355)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 4355)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 4355)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 4355)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 4355)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 4355)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 4355)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 4355)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature <= 4355)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 4355)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 4355)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 4355)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature > 4355)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature > 4355)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 4355)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 4355)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 4355)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 4355)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 4355)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 4355)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 4355)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 4786)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature <= 4786)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 4786)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 4786)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature <= 4786)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 4786)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 4786)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 4786)):
        result_gpr[2] &= ~(1 << 22)

    return result_gpr


def feature12(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4000)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 4000)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 4000)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 4000)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 4000)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature > 4000)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 2816)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 2816)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 2816)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 4000)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 4000)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 2784)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 2784)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature <= 2784)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 2784)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 2784)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 2784)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 2784)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 2784)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 2784)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 2784)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 2784)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 2784)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 2784)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 2784)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 2784)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 2784)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 2784)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 2784)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 3360)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 3360)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 3360)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature <= 3872 and feature <= 3520)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 3872 and feature > 3520)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature <= 3872)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 3872)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 3872)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 3872)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 3872)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 3872)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 3872)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 3872)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 3872)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 3872)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 3872)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 3872)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 3872)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 3872)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 3872)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 3872)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 3872)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 3872)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 3872)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 3872)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 3872)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 3872)):
        result_gpr[4] &= ~(1 << 22)

    return result_gpr


def feature13(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4294)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature > 4294)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 1850)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 1850)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 1916)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 1916)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 1916)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 1916)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 6012)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 6012)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 5549)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature <= 5549)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 5549)):
        result_gpr[7] &= ~(1 << 11)

    return result_gpr


def feature14(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2954)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 2954)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 4844)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 4844)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 4844)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 4254)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 4254)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 4254)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 4254)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature <= 5120)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 5120)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 3387)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 3387)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 2087)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 2087)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 2087)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 2087)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 2087)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 2087)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 2087)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 2087)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 2087)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 2087)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 2087)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 5474)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 5474)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 5474)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 5632)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 5632)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 5632)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 5632)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 5632)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 6105)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 6105)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 6105)):
        result_gpr[0] &= ~(1 << 9)

    return result_gpr


def feature15(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2805)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 2805)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 3473)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 3473)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 3473)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 3473)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 4096)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 4096)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 4096)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 4096)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature <= 4096)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 4096)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 4096)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 3918)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 3918)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 3918)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 3918)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 3606)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 3606)):
        result_gpr[2] &= ~(1 << 9)

    return result_gpr


def feature16(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2256)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 2256)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 2256)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 2256)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 2256)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 2256)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 2256)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 2256)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 2256)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 2256)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 2256 and feature <= 2048)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 2256 and feature > 2048)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 2256)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 2256)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 2256)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 2256)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature > 2256)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 2256)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 2256)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 2256)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 2256)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature > 2256)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 2256)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 2256)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 2256)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 2256)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 2256)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature > 2256)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 2256)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature > 2256)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature > 2256)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature > 2256)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 2256)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 2256)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 2256)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 2256)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 2256)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 2256)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 2256)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 2256)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 2256)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 2256)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 2256)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 2256)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 2256)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 2256)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 3020)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 3020)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 3367)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 3367)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 3159)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 3159)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 3992)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature <= 3992)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 3992)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 3992)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 4547 and feature <= 3298)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 4547 and feature <= 3298)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 4547 and feature <= 3298)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 4547 and feature > 3298)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature <= 4547 and feature <= 3089)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 4547 and feature > 3089)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 4547)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 4547)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 4547)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 4547)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature <= 4547)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 4547)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 4547 and feature <= 3645)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 4547 and feature <= 3645)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 4547 and feature > 3645)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 4547)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 4547)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 4547)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 4547)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 4547)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 4547)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 4547)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 4547)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 4547)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 4547)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 4547)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 4547)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 4547)):
        result_gpr[2] &= ~(1 << 7)

    return result_gpr


def feature17(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 5056 and feature <= 2240)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 5056 and feature > 2240)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 5056 and feature > 2240)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 5056 and feature <= 2496)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature <= 5056 and feature <= 2496)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 5056 and feature <= 2496)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 5056 and feature > 2496)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 0)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 5056 and feature <= 4224)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 5056 and feature > 4224)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 5056 and feature <= 3904)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 5056 and feature <= 3904)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 5056 and feature <= 3904)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 5056 and feature <= 3904)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 5056 and feature > 3904)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 5056)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature <= 5056)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature <= 5056 and feature <= 4928)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 5056 and feature <= 4928)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 5056 and feature > 4928)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 5056)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 5056)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 5056 and feature <= 4672)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature <= 5056 and feature > 4672)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature <= 5056)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature <= 5056)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature <= 5056)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature <= 5056)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 5056)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 5056)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 5056)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 5056)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 5056)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 5056)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 5056)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 5056)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 5056)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature > 5056)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 5056)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature > 5056)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 5056)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 5056)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 5056)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 5056)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 5056)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 5056)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 5056 and feature <= 5696)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 5056 and feature <= 5696)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 5056 and feature <= 5696)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 5056 and feature <= 5696)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 5056 and feature <= 5696)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 5056 and feature <= 5696)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 5056 and feature > 5696)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 5056 and feature > 5696)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 5056 and feature > 5696)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 5056)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 5056)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 5056)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 5056)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 5056)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 5056 and feature <= 5952)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature > 5056 and feature > 5952)):
        result_gpr[6] &= ~(1 << 30)

    return result_gpr


def feature18(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2744)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 2744 and feature <= 2585)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 2744 and feature > 2585)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 2744 and feature > 2585)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 2744 and feature > 2585)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 2744 and feature > 2585)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 2744 and feature > 2585)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature > 2744)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 2744)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 2744 and feature <= 3539)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature > 2744 and feature > 3539)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature > 2744 and feature > 3539)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 2744)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 2744)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 2744)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 2744)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 2744)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature > 2744)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 2744)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 2744)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 2744)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 2744)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature > 2744)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature > 2744)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 2744)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 2744)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 2744 and feature <= 4374 and feature <= 3619)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 2744 and feature <= 4374 and feature > 3619)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 2744 and feature > 4374)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 3022)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 3022)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 3022)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 3022)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 3022)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 3022)):
        result_gpr[7] &= ~(1 << 0)

    if (not(feature <= 3022 and feature <= 2585)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature <= 3022 and feature > 2585)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature <= 3022 and feature > 2585)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 3022 and feature > 2585)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature > 3022)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 3022)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 3022)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature > 3022)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 3022)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature > 3022)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 3022)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 3022)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 3022)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 3022)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 3022)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 3022)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 3022)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 3022)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 3022)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 3022)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature > 3022 and feature <= 4056)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 3022 and feature > 4056)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 3022 and feature > 4056)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 3022 and feature > 4056)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 3022)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 3022)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 3022)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 3022)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 3022)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 3022)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 3022)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 3022)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 3022)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 3022)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 3022)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 3022 and feature <= 4215)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature > 3022 and feature <= 4215)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 3022 and feature <= 4215)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 3022 and feature > 4215)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 3022 and feature > 4215)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 3022 and feature > 4215 and feature <= 5011)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 3022 and feature > 4215 and feature > 5011)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 3022 and feature > 4215 and feature > 5011)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 3022)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 3022)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 3022)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 3022)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 3022)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 3022)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 3022 and feature <= 5528)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 3022 and feature > 5528)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 3022 and feature <= 5886)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 3022 and feature <= 5886)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 3022 and feature <= 5886)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 3022 and feature <= 5886)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 3022 and feature > 5886)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 3022 and feature > 5886)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 3022 and feature > 5886)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 5528)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 5528)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 5528)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 5528)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 5528)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 5528)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 5528)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 5528)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 5528)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 5528)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature <= 5528)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 5528)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 5528)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 5528)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 5528)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature <= 5528)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature <= 5528)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature <= 5528)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature <= 5528)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 5528)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 5528)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature > 5528)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature > 5528)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 5528)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 5528)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 5528)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature > 5528)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature > 5528)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature > 5528)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature > 5528)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 5528)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 5528)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature > 5528)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 5528 and feature <= 6164)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 5528 and feature <= 6164)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 5528 and feature <= 6164)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 5528 and feature <= 6164)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 5528 and feature > 6164)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 5528 and feature > 6164)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 5528)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 5528)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 5528)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature > 5528)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 5528)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 5528)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature > 5528)):
        result_gpr[6] &= ~(1 << 30)

    return result_gpr


def feature19(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2862)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 2862)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature > 2862)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature <= 2764)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 2764)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 2961)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 2961)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 4491)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 4491)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 4491)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 4491)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 4491)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 4491)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 4491)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature > 4491)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature > 4491)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature > 4491)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature <= 4540)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 4540)):
        result_gpr[3] &= ~(1 << 22)

    return result_gpr


def feature20(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2631)):
        result_gpr[5] &= ~(1 << 31)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 2631 and feature <= 1565)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 2631 and feature <= 1565)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 2631 and feature > 1565)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 2631)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 2631)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature <= 2631 and feature <= 2298)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature <= 2631 and feature <= 2298)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature <= 2631 and feature <= 2298)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature <= 2631 and feature <= 2298)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature <= 2631 and feature <= 2298)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature <= 2631 and feature > 2298)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature <= 2631 and feature > 2298)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature <= 2631 and feature > 2298)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature <= 2631 and feature > 2298)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 2631 and feature > 2298)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature <= 2631 and feature > 2298)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 2631)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature <= 2631)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 2631)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 2631)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 2631)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 2631)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 2631)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature <= 2631)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 2631)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 2631)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 2631)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature <= 2631)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature <= 2631)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature <= 2631)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 2631)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature > 2631 and feature <= 3763)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 2631 and feature <= 3763)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 2631 and feature <= 3763)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature > 2631 and feature <= 3763)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 2631 and feature <= 3763)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 2631 and feature <= 3763)):
        result_gpr[7] &= ~(1 << 0)

    if (not(feature > 2631 and feature > 3763)):
        result_gpr[7] &= ~(1 << 9)

    if (not(feature > 2631 and feature > 3763)):
        result_gpr[7] &= ~(1 << 16)

    if (not(feature > 2631 and feature > 3763)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 2631 and feature > 3763)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 2631)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature > 2631)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 2631)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature > 2631)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 2631)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 2631)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 2631)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 2631)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 2631)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 2631)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 2631)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 2631)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 2631)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 2631)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 2631)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 2631)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 2631)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 2631)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 2631)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 2631)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 2631)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 2631)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 2631)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 2631)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 2631)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 2631)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 2631)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 2631)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 2631)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 2631)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 2631)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 2631)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 2631)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 2631)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 2631)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 2631 and feature <= 3663)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 2631 and feature > 3663)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 2897)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 2897)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 4629)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 4629)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 4629)):
        result_gpr[4] &= ~(1 << 22)

    return result_gpr


def feature21(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2332)):
        result_gpr[7] &= ~(1 << 2)

    if (not(feature > 2332)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature > 2332)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 4285)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 4285)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature <= 4285)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature <= 4285)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature <= 4285)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature <= 4285)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature <= 4285)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature <= 4285)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature <= 4285)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 4285)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 4285)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 4285)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 4285)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 4285)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 4285 and feature <= 3277)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4285 and feature > 3277)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 4285)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 4285)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 4285)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 4285)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 4285)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 4285)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 4285)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 4285)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 4285)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 4285)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 4285)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 4285)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 4285)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 4285)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature <= 4285)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature <= 4285)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 4285)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 4285)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature <= 4285)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature > 4285)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 4285)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature > 4285)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 4285)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 4285)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature > 4285)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature > 4285)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature > 4285)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature > 4285)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 4285)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 4285)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 4285)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 4285)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 4285)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 4285)):
        result_gpr[7] &= ~(1 << 6)

    if (not(feature > 4285 and feature <= 4789)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 4285 and feature <= 4789)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 4285 and feature > 4789)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 4285 and feature > 4789)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 4285 and feature > 4789)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 4285 and feature > 4789)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 4285)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 4285)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 4285)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 4285)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 4285)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 4285)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 4285)):
        result_gpr[0] &= ~(1 << 20)

    if (not(feature > 4285)):
        result_gpr[0] &= ~(1 << 21)

    if (not(feature > 4285)):
        result_gpr[4] &= ~(1 << 31)

    if (not(feature > 4285)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 24)

    if (not(feature > 4285)):
        result_gpr[5] &= ~(1 << 25)

    if (not(feature > 4285)):
        result_gpr[7] &= ~(1 << 11)

    if (not(feature <= 5041)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 5041)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 5041)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 5041)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 5608)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 5608)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature > 5608)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 5608)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 5608)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 5608)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 5356)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 5356)):
        result_gpr[0] &= ~(1 << 27)

    return result_gpr


def feature22(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2903)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature > 2903)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 3460)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 3460)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 4573)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 4573)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 5886)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 5886)):
        result_gpr[3] &= ~(1 << 2)

    return result_gpr


def feature23(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2048)):
        result_gpr[0] &= ~(1 << 22)

    if (not(feature > 2048)):
        result_gpr[0] &= ~(1 << 23)

    if (not(feature <= 3322 and feature <= 1775)):
        result_gpr[7] &= ~(1 << 5)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature <= 3322 and feature > 1775)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature > 3322)):
        result_gpr[5] &= ~(1 << 12)

    if (not(feature > 3322)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 3322)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature > 3322 and feature <= 3459)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature > 3322 and feature > 3459)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 3322 and feature > 3459)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 3322)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 3322)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 3322 and feature <= 4324)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 3322 and feature > 4324)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 3322)):
        result_gpr[7] &= ~(1 << 22)

    if (not(feature > 3322)):
        result_gpr[4] &= ~(1 << 20)

    if (not(feature > 3322)):
        result_gpr[4] &= ~(1 << 21)

    if (not(feature > 3322)):
        result_gpr[6] &= ~(1 << 11)

    if (not(feature > 3322)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 3322)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 3231)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 3231)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 3231)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 3231)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 3231)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 3231)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 4961)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 4961)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 4961)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 4961)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature > 4961)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 4961)):
        result_gpr[2] &= ~(1 << 3)

    return result_gpr


def feature24(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2916)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 2916)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 2916)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 2916)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 2916)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 2949)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 2949)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 2949)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature <= 2490)):
        result_gpr[0] &= ~(1 << 18)

    if (not(feature > 2490)):
        result_gpr[0] &= ~(1 << 19)

    if (not(feature <= 2785)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature > 2785)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 3539)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 3539)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 3867)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 3867)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 3867)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 3867)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 3867)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 3867)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 3834)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 3834)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 2884)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 2884)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 2884)):
        result_gpr[2] &= ~(1 << 25)

    return result_gpr


def feature25(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3776)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 3776)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 4544)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 4544)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 4544)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 4544)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 4544)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 4544)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature <= 4544)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature <= 4544)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature <= 4544)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4544)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 4544)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 4544)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 4544)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 4544)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 4544)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 4544)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 4544)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 4544)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 4544)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 4544 and feature <= 4288)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 4544 and feature > 4288)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 4544)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature <= 4544)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature > 4544 and feature <= 5248)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature > 4544 and feature > 5248)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 4544 and feature > 5248)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 4544)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 4544)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 4544)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 4544)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 4544)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 4544)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 4544)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 4544 and feature <= 5760)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 4544 and feature <= 5760)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 4544 and feature > 5760)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 4544 and feature > 5760)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 4160)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature <= 4160)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 4160)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 4160)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 4160)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature > 4160)):
        result_gpr[7] &= ~(1 << 10)

    return result_gpr


def feature26(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4608)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 4608)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 4608)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature > 4608 and feature <= 5159)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature > 4608 and feature <= 5159)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature > 4608 and feature > 5159)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 4608 and feature > 5159)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 4608 and feature > 5159)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature <= 2993)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature <= 2993)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature <= 2993)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature <= 2993)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 2993)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature > 2993)):
        result_gpr[4] &= ~(1 << 27)

    if (not(feature > 2993)):
        result_gpr[4] &= ~(1 << 28)

    if (not(feature > 2993)):
        result_gpr[7] &= ~(1 << 7)

    if (not(feature > 2993)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature > 2993)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 2993)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature > 2993)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 2993)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 2993 and feature <= 3623)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature > 2993 and feature > 3623)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 2993)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 2993)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 2993)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 2993)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 2993)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 2993)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 2993)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 2993)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 5829)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 5829)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 5396)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 5396)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 4529)):
        result_gpr[6] &= ~(1 << 31)

    if (not(feature <= 4529)):
        result_gpr[5] &= ~(1 << 0)

    if (not(feature <= 4529)):
        result_gpr[7] &= ~(1 << 13)

    if (not(feature <= 4529)):
        result_gpr[4] &= ~(1 << 25)

    if (not(feature <= 4529)):
        result_gpr[4] &= ~(1 << 26)

    if (not(feature <= 4529)):
        result_gpr[7] &= ~(1 << 10)

    if (not(feature > 4529)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 4529)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 4529)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 4529)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 4923)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature > 4923)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 4923)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 4923)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 5947)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 5947)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 5947)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 5947)):
        result_gpr[3] &= ~(1 << 20)

    return result_gpr


def feature27(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2495)):
        result_gpr[6] &= ~(1 << 1)

    if (not(feature <= 2495)):
        result_gpr[6] &= ~(1 << 2)

    if (not(feature <= 2495)):
        result_gpr[6] &= ~(1 << 0)

    if (not(feature > 2495)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 2495)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 2495)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 2495)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 2495)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 2495)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature > 2495)):
        result_gpr[5] &= ~(1 << 3)

    if (not(feature > 2495)):
        result_gpr[5] &= ~(1 << 4)

    if (not(feature <= 3390)):
        result_gpr[6] &= ~(1 << 9)

    if (not(feature <= 3390)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 3390)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 3390)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature > 3390)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 3390)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 3390)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 3955)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 3955)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 2966)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 2966)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 2966)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature > 2966)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature <= 3437)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 3437)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 3437)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 3908)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature <= 3908)):
        result_gpr[6] &= ~(1 << 19)

    if (not(feature > 3908 and feature <= 5038)):
        result_gpr[5] &= ~(1 << 5)

    if (not(feature > 3908 and feature > 5038)):
        result_gpr[5] &= ~(1 << 6)

    if (not(feature > 3908)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 3908)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 3908)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 3908)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 3908)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 3719)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 3719)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 4567)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 4567)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 4567)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 4567)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 4567)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature > 4567)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature > 4567)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature > 4567)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 4567)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 4567)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 4567)):
        result_gpr[4] &= ~(1 << 16)

    return result_gpr


def feature28(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2526)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature > 2526)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 3721)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 3721)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 2287)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 2287)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 4540)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 4540)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 4540)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 4540)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 4540)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 4540)):
        result_gpr[7] &= ~(1 << 0)

    if (not(feature <= 3721)):
        result_gpr[5] &= ~(1 << 26)

    if (not(feature <= 3721)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 3721)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 3721)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 3721)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 3721)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature <= 3721)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 3721)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 3721)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 3721)):
        result_gpr[0] &= ~(1 << 17)

    if (not(feature > 3721)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature > 3721)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature > 3721)):
        result_gpr[6] &= ~(1 << 26)

    if (not(feature > 3721)):
        result_gpr[6] &= ~(1 << 27)

    if (not(feature <= 2970)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 2970)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 2970)):
        result_gpr[6] &= ~(1 << 14)

    if (not(feature <= 2970)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 2970)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 2970)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 2970)):
        result_gpr[5] &= ~(1 << 7)

    if (not(feature > 2970)):
        result_gpr[5] &= ~(1 << 8)

    if (not(feature > 2970)):
        result_gpr[5] &= ~(1 << 9)

    if (not(feature > 2970)):
        result_gpr[5] &= ~(1 << 10)

    if (not(feature <= 3550)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 3550)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 1980)):
        result_gpr[6] &= ~(1 << 18)

    if (not(feature > 1980)):
        result_gpr[6] &= ~(1 << 19)

    return result_gpr


def feature29(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3592)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 3592)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 3592)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4789)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature > 4789)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 1323)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 1323)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 1323)):
        result_gpr[6] &= ~(1 << 17)

    if (not(feature <= 4789)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 4789)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 4789)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 4789)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature > 4789)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 4789)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 4789)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 3970)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 3970)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 3970)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature > 3970)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature > 3970)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature > 3970)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 5797)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature > 5797)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 5797)):
        result_gpr[2] &= ~(1 << 7)

    return result_gpr


def feature30(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3082)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 3082)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 3082)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 3082)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 3082)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 3082)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 3082)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 3082)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 3082)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 3082)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 6043)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature <= 6043)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 6043)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 6043)):
        result_gpr[6] &= ~(1 << 25)

    if (not(feature <= 4907)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 4907)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 4907)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 4907)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature > 4907)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature > 4907)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature > 4907)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature > 4907)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature > 4907)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature > 4907)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature > 4907)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature > 4907)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 4907)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 5880)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature > 5880)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 5799 and feature <= 5556)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 5799 and feature > 5556)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature > 5799)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature > 5799)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 5799)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 4583)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature > 4583)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 4583)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 4583)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 4583)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 4583)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 4583)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature > 4583)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature > 4583)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature > 4583)):
        result_gpr[2] &= ~(1 << 25)

    return result_gpr


def feature31(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2458)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 2458)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 2458)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature > 2458)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 1854)):
        result_gpr[7] &= ~(1 << 15)

    if (not(feature > 1854)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature > 1854)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 1854)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature > 1854)):
        result_gpr[4] &= ~(1 << 29)

    if (not(feature > 1854)):
        result_gpr[4] &= ~(1 << 30)

    if (not(feature > 1854)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 1854)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 1854)):
        result_gpr[6] &= ~(1 << 7)

    if (not(feature > 1854)):
        result_gpr[4] &= ~(1 << 7)

    if (not(feature > 1854)):
        result_gpr[4] &= ~(1 << 8)

    if (not(feature <= 4915)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 4915)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 4915)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 4915)):
        result_gpr[6] &= ~(1 << 12)

    if (not(feature <= 4915)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 4915)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 4915)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 4915)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 4915)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 4915)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 4915)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 4915)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature <= 4915)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature > 4915)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 4915)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 4915)):
        result_gpr[5] &= ~(1 << 11)

    if (not(feature <= 6985)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 6985)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature <= 3665)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 3665)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 3665)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 3665)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 3665)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 3665)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 3665)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 3665)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 3665)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 3665)):
        result_gpr[2] &= ~(1 << 22)

    return result_gpr


def feature32(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1856)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 1856)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature <= 3168)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature <= 3168)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature <= 3168)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature <= 3168)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 3168)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 3168)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature <= 3168)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 3168)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 3168)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature > 3168)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature > 3168)):
        result_gpr[6] &= ~(1 << 16)

    if (not(feature <= 1952)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 1952)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 3680)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 3680)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 3680)):
        result_gpr[4] &= ~(1 << 9)

    if (not(feature <= 3680)):
        result_gpr[4] &= ~(1 << 10)

    if (not(feature > 3680)):
        result_gpr[7] &= ~(1 << 1)

    if (not(feature <= 2816)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 2816)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 2336)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature > 2336)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature <= 3296)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 3296)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 3296)):
        result_gpr[6] &= ~(1 << 10)

    if (not(feature > 3296)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 3296)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 3392)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature > 3392)):
        result_gpr[5] &= ~(1 << 2)

    return result_gpr


def feature33(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3328)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 3328)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 2112)):
        result_gpr[7] &= ~(1 << 8)

    if (not(feature > 2112)):
        result_gpr[4] &= ~(1 << 5)

    if (not(feature > 2112)):
        result_gpr[4] &= ~(1 << 6)

    if (not(feature > 2112)):
        result_gpr[5] &= ~(1 << 20)

    if (not(feature > 2112)):
        result_gpr[5] &= ~(1 << 21)

    if (not(feature > 2112)):
        result_gpr[6] &= ~(1 << 24)

    if (not(feature > 2112)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 2112)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 2112)):
        result_gpr[6] &= ~(1 << 5)

    if (not(feature <= 3904)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 3904)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 3904)):
        result_gpr[6] &= ~(1 << 15)

    if (not(feature <= 4032)):
        result_gpr[5] &= ~(1 << 22)

    if (not(feature <= 4032)):
        result_gpr[5] &= ~(1 << 23)

    if (not(feature > 4032)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 4032)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 4032)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 4032)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 4032)):
        result_gpr[7] &= ~(1 << 21)

    if (not(feature > 4032)):
        result_gpr[7] &= ~(1 << 19)

    if (not(feature <= 5824)):
        result_gpr[6] &= ~(1 << 4)

    if (not(feature <= 5824)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature <= 5824)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 5824)):
        result_gpr[6] &= ~(1 << 3)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature <= 5824)):
        result_gpr[5] &= ~(1 << 14)

    if (not(feature <= 5824)):
        result_gpr[5] &= ~(1 << 15)

    if (not(feature <= 5824)):
        result_gpr[6] &= ~(1 << 8)

    if (not(feature <= 5824)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 5824)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 13)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 17)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 18)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 14)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 5824)):
        result_gpr[6] &= ~(1 << 6)

    if (not(feature <= 5824)):
        result_gpr[5] &= ~(1 << 30)

    if (not(feature <= 5824 and feature <= 5568)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 5824 and feature > 5568)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 5824 and feature > 5568)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 5824 and feature > 5568)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 5824)):
        result_gpr[5] &= ~(1 << 29)

    if (not(feature <= 5824)):
        result_gpr[7] &= ~(1 << 28)

    if (not(feature <= 5824)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 5824)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 5824)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 5824)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature <= 5824)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature <= 5824)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 23)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 24)

    if (not(feature <= 5824)):
        result_gpr[4] &= ~(1 << 22)

    if (not(feature <= 5824)):
        result_gpr[7] &= ~(1 << 3)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 5824)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 5824)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 5824)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 5824)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 5824 and feature <= 6336)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 5824 and feature <= 6336)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 5824 and feature > 6336)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 5824)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 5824)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature > 5824)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 5824)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature > 5824)):
        result_gpr[6] &= ~(1 << 30)

    return result_gpr


def feature34(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 2823)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 2823)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 2823)):
        result_gpr[7] &= ~(1 << 14)

    if (not(feature <= 2823)):
        result_gpr[5] &= ~(1 << 18)

    if (not(feature <= 2823)):
        result_gpr[5] &= ~(1 << 19)

    if (not(feature > 2823)):
        result_gpr[7] &= ~(1 << 20)

    if (not(feature > 2823)):
        result_gpr[5] &= ~(1 << 28)

    if (not(feature > 2823)):
        result_gpr[4] &= ~(1 << 0)

    if (not(feature > 2823)):
        result_gpr[4] &= ~(1 << 1)

    if (not(feature > 2823)):
        result_gpr[5] &= ~(1 << 27)

    if (not(feature > 2823)):
        result_gpr[5] &= ~(1 << 16)

    if (not(feature > 2823)):
        result_gpr[5] &= ~(1 << 17)

    if (not(feature > 2823)):
        result_gpr[4] &= ~(1 << 11)

    if (not(feature > 2823)):
        result_gpr[4] &= ~(1 << 12)

    if (not(feature > 2823)):
        result_gpr[7] &= ~(1 << 27)

    if (not(feature > 2823)):
        result_gpr[7] &= ~(1 << 26)

    if (not(feature <= 4215)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 4215)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 4215)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4215)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature > 4215)):
        result_gpr[0] &= ~(1 << 24)

    if (not(feature > 4215)):
        result_gpr[0] &= ~(1 << 25)

    if (not(feature <= 5369)):
        result_gpr[6] &= ~(1 << 28)

    if (not(feature <= 5369)):
        result_gpr[6] &= ~(1 << 29)

    if (not(feature > 5369)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 5369)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 2585)):
        result_gpr[7] &= ~(1 << 4)

    if (not(feature > 2585)):
        result_gpr[7] &= ~(1 << 30)

    if (not(feature > 2585)):
        result_gpr[7] &= ~(1 << 31)

    if (not(feature > 2585)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 2585)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 5130)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 5130)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 5846)):
        result_gpr[4] &= ~(1 << 15)

    if (not(feature > 5846)):
        result_gpr[4] &= ~(1 << 16)

    if (not(feature <= 5090)):
        result_gpr[4] &= ~(1 << 19)

    if (not(feature > 5090)):
        result_gpr[0] &= ~(1 << 26)

    if (not(feature > 5090)):
        result_gpr[0] &= ~(1 << 27)

    if (not(feature <= 6402)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature <= 6402)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 6402)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 6402)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 6402)):
        result_gpr[3] &= ~(1 << 18)

    return result_gpr


def feature35(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 3277)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 3277)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 5648)):
        result_gpr[6] &= ~(1 << 21)

    if (not(feature <= 5648)):
        result_gpr[6] &= ~(1 << 22)

    if (not(feature <= 5648)):
        result_gpr[7] &= ~(1 << 25)

    if (not(feature > 5648)):
        result_gpr[0] &= ~(1 << 28)

    if (not(feature > 5648)):
        result_gpr[0] &= ~(1 << 29)

    if (not(feature <= 4915)):
        result_gpr[7] &= ~(1 << 23)

    if (not(feature <= 4915)):
        result_gpr[7] &= ~(1 << 24)

    if (not(feature <= 4915)):
        result_gpr[4] &= ~(1 << 3)

    if (not(feature <= 4915)):
        result_gpr[4] &= ~(1 << 4)

    if (not(feature <= 4915)):
        result_gpr[4] &= ~(1 << 2)

    if (not(feature <= 4915)):
        result_gpr[5] &= ~(1 << 1)

    if (not(feature <= 4915)):
        result_gpr[5] &= ~(1 << 2)

    if (not(feature <= 4915)):
        result_gpr[6] &= ~(1 << 13)

    if (not(feature > 4915)):
        result_gpr[7] &= ~(1 << 17)

    if (not(feature <= 5476)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 5476)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature <= 4570)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature > 4570)):
        result_gpr[2] &= ~(1 << 22)

    return result_gpr


def feature36(feature):
    result_gpr = [0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 1312)):
        result_gpr[7] &= ~(1 << 12)

    if (not(feature > 1312)):
        result_gpr[0] &= ~(1 << 30)

    if (not(feature > 1312)):
        result_gpr[0] &= ~(1 << 31)

    if (not(feature > 1312)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 1312)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 1312 and feature <= 2848)):
        result_gpr[6] &= ~(1 << 23)

    if (not(feature > 1312 and feature > 2848)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 1312 and feature > 2848)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 2080)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 2080)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature > 2080)):
        result_gpr[7] &= ~(1 << 18)

    if (not(feature <= 4032)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 4032)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 4032)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 4032)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 3520)):
        result_gpr[5] &= ~(1 << 13)

    if (not(feature > 3520)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 3520)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 2976)):
        result_gpr[7] &= ~(1 << 29)

    if (not(feature > 2976)):
        result_gpr[6] &= ~(1 << 20)

    if (not(feature > 2976)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 2976)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 2976)):
        result_gpr[3] &= ~(1 << 11)

    return result_gpr


In [ ]:
def dtree_aggr(result1, result2, result3, result4, result5, result6, result7, result8, result9, result10, \
              result11, result12, result13, result14, result15, result16, result17, result18, result19, result20, \
              result21, result22, result23, result24, result25, result26, result27, result28, result29, result30, \
              result31, result32, result33, result34, result35, result36):
    
    path_class = [2, 1, 1, 1, 2, 3, 2, 5, 1, 2, 7, 3, 5, 7, 5, 5, 2, 7, 7, 1, 5, 4, 7, 7, 7, 4, 4, 4, 7, 7, 1, 
                  5, 2, 5, 7, 4, 7, 5, 3, 7, 4, 4, 4, 3, 7, 4, 7, 7, 7, 5, 3, 7, 1, 4, 7, 4, 4, 1, 3, 1, 7, 4, 
                  3, 2, 4, 3, 4, 2, 7, 4, 3, 4, 3, 1, 3, 3, 7, 4, 4, 3, 5, 1, 7, 2, 7, 1, 2, 1, 3, 1, 5, 5, 5, 
                  3, 3, 2, 4, 7, 7, 4, 3, 7, 5, 7, 4, 5, 2, 4, 7, 3, 3, 3, 4, 3, 4, 3, 4, 7, 1, 3, 1, 5, 4, 1, 
                  4, 7, 4, 5, 1, 5, 1, 5, 5, 7, 5, 4, 7, 7, 4, 4, 1, 5, 5, 4, 7, 3, 3, 3, 3, 3, 4, 7, 2, 7, 5, 
                  4, 3, 4, 3, 4, 5, 1, 3, 7, 2, 5, 3, 1, 7, 3, 1, 1, 5, 2, 4, 3, 7, 4, 5, 1, 4, 4, 3, 4, 4, 3, 
                  1, 4, 5, 4, 7, 4, 3, 4, 5, 7, 5, 7, 5, 1, 5, 1, 5, 7, 4, 4, 4, 3, 7, 3, 7, 4, 1, 3, 4, 7, 2, 
                  4, 7, 3, 7, 2, 4, 3, 7, 3, 3, 3, 7, 4, 1, 5, 4, 3, 7, 4, 3, 7, 3, 4, 4, 5, 7, 4, 1, 3, 7, 1, 
                  4, 7, 1, 4, 1, 5, 2, 4,]
    
    result_cpy = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    result_cpy[0] = result1[0] & result2[0] & result3[0] & result4[0] & result5[0] & result6[0] & result7[0] \
    & result8[0] & result9[0] & result10[0] & result11[0] & result12[0] & result13[0] & result14[0] \
    & result15[0] & result16[0] & result17[0] & result18[0] & result19[0] & result20[0] & result21[0] \
    & result22[0] & result23[0] & result24[0] & result25[0] & result26[0] & result27[0] & result28[0] \
    & result29[0] & result30[0] & result31[0] & result32[0] & result33[0] & result34[0] & result35[0] \
    & result36[0]
    
    result_cpy[1] = result1[1] & result2[1] & result3[1] & result4[1] & result5[1] & result6[1] & result7[1] \
    & result8[1] & result9[1] & result10[1] & result11[1] & result12[1] & result13[1] & result14[1] \
    & result15[1] & result16[1] & result17[1] & result18[1] & result19[1] & result20[1] & result21[1] \
    & result22[1] & result23[1] & result24[1] & result25[1] & result26[1] & result27[1] & result28[1] \
    & result29[1] & result30[1] & result31[1] & result32[1] & result33[1] & result34[1] & result35[1] \
    & result36[1]
    
    result_cpy[2] = result1[2] & result2[2] & result3[2] & result4[2] & result5[2] & result6[2] & result7[2] \
    & result8[2] & result9[2] & result10[2] & result11[2] & result12[2] & result13[2] & result14[2] \
    & result15[2] & result16[2] & result17[2] & result18[2] & result19[2] & result20[2] & result21[2] \
    & result22[2] & result23[2] & result24[2] & result25[2] & result26[2] & result27[2] & result28[2] \
    & result29[2] & result30[2] & result31[2] & result32[2] & result33[2] & result34[2] & result35[2] \
    & result36[2]    
    
    result_cpy[3] = result1[3] & result2[3] & result3[3] & result4[3] & result5[3] & result6[3] & result7[3] \
    & result8[3] & result9[3] & result10[3] & result11[3] & result12[3] & result13[3] & result14[3] \
    & result15[3] & result16[3] & result17[3] & result18[3] & result19[3] & result20[3] & result21[3] \
    & result22[3] & result23[3] & result24[3] & result25[3] & result26[3] & result27[3] & result28[3] \
    & result29[3] & result30[3] & result31[3] & result32[3] & result33[3] & result34[3] & result35[3] \
    & result36[3]    
    
    result_cpy[4] = result1[4] & result2[4] & result3[4] & result4[4] & result5[4] & result6[4] & result7[4] \
    & result8[4] & result9[4] & result10[4] & result11[4] & result12[4] & result13[4] & result14[4] \
    & result15[4] & result16[4] & result17[4] & result18[4] & result19[4] & result20[4] & result21[4] \
    & result22[4] & result23[4] & result24[4] & result25[4] & result26[4] & result27[4] & result28[4] \
    & result29[4] & result30[4] & result31[4] & result32[4] & result33[4] & result34[4] & result35[4] \
    & result36[4]
    
    result_cpy[5] = result1[5] & result2[5] & result3[5] & result4[5] & result5[5] & result6[5] & result7[5] \
    & result8[5] & result9[5] & result10[5] & result11[5] & result12[5] & result13[5] & result14[5] \
    & result15[5] & result16[5] & result17[5] & result18[5] & result19[5] & result20[5] & result21[5] \
    & result22[5] & result23[5] & result24[5] & result25[5] & result26[5] & result27[5] & result28[5] \
    & result29[5] & result30[5] & result31[5] & result32[5] & result33[5] & result34[5] & result35[5] \
    & result36[5]
    
    result_cpy[6] = result1[6] & result2[6] & result3[6] & result4[6] & result5[6] & result6[6] & result7[6] \
    & result8[6] & result9[6] & result10[6] & result11[6] & result12[6] & result13[6] & result14[6] \
    & result15[6] & result16[6] & result17[6] & result18[6] & result19[6] & result20[6] & result21[6] \
    & result22[6] & result23[6] & result24[6] & result25[6] & result26[6] & result27[6] & result28[6] \
    & result29[6] & result30[6] & result31[6] & result32[6] & result33[6] & result34[6] & result35[6] \
    & result36[6]
    
    result_cpy[7] = result1[7] & result2[7] & result3[7] & result4[7] & result5[7] & result6[7] & result7[7] \
    & result8[7] & result9[7] & result10[7] & result11[7] & result12[7] & result13[7] & result14[7] \
    & result15[7] & result16[7] & result17[7] & result18[7] & result19[7] & result20[7] & result21[7] \
    & result22[7] & result23[7] & result24[7] & result25[7] & result26[7] & result27[7] & result28[7] \
    & result29[7] & result30[7] & result31[7] & result32[7] & result33[7] & result34[7] & result35[7] \
    & result36[7]
    
    
    add_on = 0       
    if (result_cpy[7] != 0):
        add_on = 0
        result_cpy[0] = result_cpy[7]        
    elif (result_cpy[6] != 0):
        add_on = 32
        result_cpy[0] = result_cpy[6]
    elif (result_cpy[5] != 0):
        add_on = 64
        result_cpy[0] = result_cpy[5]        
    elif (result_cpy[4] != 0):
        add_on = 96
        result_cpy[0] = result_cpy[4]        
    elif (result_cpy[3] != 0):
        add_on = 128
        result_cpy[0] = result_cpy[3]
    elif (result_cpy[2] != 0):
        add_on = 160
        result_cpy[0] = result_cpy[2]        
    elif (result_cpy[1] != 0):
        add_on = 192
        result_cpy[0] = result_cpy[1]
    else:
        add_on = 224
        result_cpy[0] = result_cpy[0]

        
    index = 0
    result_cpy[0] >>= 1
    while(result_cpy[0] != 0):
        result_cpy[0] >>= 1
        index += 1
        
    return path_class[index+add_on]
        

In [ ]:
clf_preds = clf.predict(test_features)
net_pred = []

In [ ]:
for i in range(len(test_features)):
    if_ex = [num_translate1(feat, total_bits, dec_bits) for feat in test_features[i]]

    
    result1 = feature1(if_ex[0])
    result2 = feature2(if_ex[1])
    result3 = feature3(if_ex[2])
    result4 = feature4(if_ex[3])
    result5 = feature5(if_ex[4])
    result6 = feature6(if_ex[5])
    result7 = feature7(if_ex[6])
    result8 = feature8(if_ex[7])
    result9 = feature9(if_ex[8])
    result10 = feature10(if_ex[9])
    
    result11 = feature11(if_ex[10])
    result12 = feature12(if_ex[11])
    result13 = feature13(if_ex[12])
    result14 = feature14(if_ex[13])
    result15 = feature15(if_ex[14])
    result16 = feature16(if_ex[15])
    result17 = feature17(if_ex[16])
    result18 = feature18(if_ex[17])
    result19 = feature19(if_ex[18])
    result20 = feature20(if_ex[19])
    
    result21 = feature21(if_ex[20])
    result22 = feature22(if_ex[21])
    result23 = feature23(if_ex[22])
    result24 = feature24(if_ex[23])
    result25 = feature25(if_ex[24])
    result26 = feature26(if_ex[25])
    result27 = feature27(if_ex[26])
    result28 = feature28(if_ex[27])
    result29 = feature29(if_ex[28])
    result30 = feature30(if_ex[29])
    
    result31 = feature31(if_ex[30])
    result32 = feature32(if_ex[31])
    result33 = feature33(if_ex[32])
    result34 = feature34(if_ex[33])
    result35 = feature35(if_ex[34])
    result36 = feature36(if_ex[35])  
    
    if_pred = dtree_aggr(result1, result2, result3, result4, result5, result6, result7, result8, result9, result10, \
                         result11, result12, result13, result14, result15, result16, result17, result18, result19, result20, \
                         result21, result22, result23, result24, result25, result26, result27, result28, result29, result30, \
                         result31, result32, result33, result34, result35, result36)  
    
    net_pred.append(if_pred)
    

In [ ]:
print(accuracy_score(test_labels, net_pred) * 100)
print(f1_score(test_labels, net_pred, average='macro') * 100)
print(precision_score(test_labels, net_pred, average='macro') * 100)
print(recall_score(test_labels, net_pred, average='macro') * 100)